# Hdfs remote connect 

In [1]:
#!pip install pyspark
# pyspark가 설치된 경로를 확인해서 SPARK_HOME 설정 

In [1]:
# !pip install findspark
# !pwd

In [1]:
# import libraries 
import socket
import sys
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
#import findspark
import time 

In [2]:
# 전역변수 설정 
# local mode 
SPARK_LOCAL_MASTER = "local[3]"
# client mode 
SPARK_CLUSTER_MASTER = "spark://34.125.237.158/:7077" 
SPARK_APP_NAME = "DATA-Preparation"
HOST_NAME = socket.gethostname()
PORT = 9999

# Define path
DATA_ROOT = f'{os.getcwd()}/data'
DATA_PATH = f'{DATA_ROOT}'

In [3]:
#os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64/'
# os.environ['PATH'] = '/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/lib/jvm/java-8-openjdk-amd64/bin/'
#os.environ['SPARK_HOME'] = '/usr/local'
#os.environ['SPARK_HOME']='/usr/local/lib/python3.6/dist-packages/pyspark'
# os.environ['HADOOP_HOME'] = '/hadoop-3.2.2'
# os.environ['hadoop.home.dir'] = '/hadoop-3.2.2/bin'
# os.environ['CLASSPATH'] = '$CLASSPATH:/hadoop-3.2.2/spark-3.2.0-bin-hadoop3.2.tar'
#findspark.init()
print(os.getenv('JAVA_HOME'))
print(os.getenv('SPARK_HOME'))

None
None


In [5]:
print(os.getenv('PATH'))
#findspark

/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/lib/jvm/java-8-openjdk-amd64/bin/


In [4]:
# Spark session 생성 메서드 
def init_remote_session():
    #SPARK_CLUSTER_MASTER = "spark://34.64.108.172:7077" 
    spark = SparkSession.builder.master(SPARK_CLUSTER_MASTER).appName(SPARK_APP_NAME).getOrCreate()
    return spark
    
# local mode 실행 시 메모리를 확장해야 하는 경우 있어서, conf에 memory 변경 추가  
def init_local_session():
    #SPARK_LOCAL_MASTER = "local[3]"
    spark = SparkSession.builder.master(SPARK_LOCAL_MASTER).appName(SPARK_APP_NAME).getOrCreate()
    default_conf = spark.sparkContext._conf#.getAll()
    print(f'Old Conf : {default_conf.getAll()}')
    conf = spark.sparkContext._conf.setAll([
        ('spark.executor.instances', 1)
        #, ('spark.driver.memory', '12g'), ('spark.executor.memory', '8g'), ('spark.driver.maxResultSize', '8g')
        , ('spark.driver.allowMultipleContexts', 'true'), ('spark.sql.shuffle.partitions', 8)
        ##,('spark.memory.offHeap.enabled', True), ('spark.memory.offHeap.size', '8g')
    ])
    spark.sparkContext.stop()
    
    spark = SparkSession.builder.master(SPARK_LOCAL_MASTER).appName(SPARK_APP_NAME).config(conf=default_conf).getOrCreate()
    new_conf = spark.sparkContext._conf
    print(f'Updated Conf : {new_conf.getAll()}')
    return spark

In [5]:
!java -version

openjdk version "1.8.0_312"
OpenJDK Runtime Environment (build 1.8.0_312-8u312-b07-0ubuntu1~18.04-b07)
OpenJDK 64-Bit Server VM (build 25.312-b07, mixed mode)


In [6]:
# 스파크 생성 
spark = init_local_session()
# 스파크 클러스터를 실행해 두었다고 가정
#spark = init_remote_session()
sc = spark.sparkContext
sc

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


Old Conf : [('spark.sql.warehouse.dir', 'file:/notebooks/spark-warehouse'), ('spark.app.name', 'DATA-Preparation'), ('spark.rdd.compress', 'True'), ('spark.driver.host', 'spark-client'), ('spark.app.id', 'local-1644828272175'), ('spark.master', 'local[3]'), ('spark.serializer.objectStreamReset', '100'), ('spark.submit.pyFiles', ''), ('spark.executor.id', 'driver'), ('spark.submit.deployMode', 'client'), ('spark.app.startTime', '1644828270976'), ('spark.driver.port', '40973'), ('spark.ui.showConsoleProgress', 'true')]
Updated Conf : [('spark.sql.warehouse.dir', 'file:/notebooks/spark-warehouse'), ('spark.driver.host', 'spark-client'), ('spark.master', 'local[3]'), ('spark.driver.allowMultipleContexts', 'true'), ('spark.executor.id', 'driver'), ('spark.app.id', 'local-1644828274138'), ('spark.driver.port', '40973'), ('spark.app.startTime', '1644828274070'), ('spark.app.name', 'DATA-Preparation'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.execu

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


<SparkContext master=local[3] appName=DATA-Preparation>

# HDFS Run
Spark Driver와 다른 위치에서 Hadoop cluster를 실행하고, 이를 연결해서 사용해 본다.  
172.17.0.3:9000 에 Hadoop 머신을 실행해 둔다.  

In [8]:
spark.read.text("hdfs://172.17.0.3:9000/user/root/input").show()

Py4JJavaError: An error occurred while calling o170.text.
: java.net.ConnectException: Call From spark-client/172.17.0.2 to 172.17.0.3:9000 failed on connection exception: java.net.ConnectException: Connection refused; For more details see:  http://wiki.apache.org/hadoop/ConnectionRefused
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at org.apache.hadoop.net.NetUtils.wrapWithMessage(NetUtils.java:913)
	at org.apache.hadoop.net.NetUtils.wrapException(NetUtils.java:828)
	at org.apache.hadoop.ipc.Client.getRpcResponse(Client.java:1577)
	at org.apache.hadoop.ipc.Client.call(Client.java:1519)
	at org.apache.hadoop.ipc.Client.call(Client.java:1416)
	at org.apache.hadoop.ipc.ProtobufRpcEngine2$Invok

위에서 hdfs 원격 read 작동을 확인하면, 샘플 데이터를 생성해 hdfs에 기록하고 읽어본다.  

In [11]:
# channel, day, seg, setop, 1000
FILE_FORMAT = 'parquet'
#FILE_FORMAT = 'csv'

# setop data n 개 만들기
def create_setops():
    setop_count = 10000
    setop_name = ['ST_A', 'ST_B', 'ST_C', 'ST_D', 'ST_E']
    setops = []
    for s in setop_name:
        for i in range(0, int(setop_count/len(setop_name))):
            setops.append(f'{s}_{i:03d}')
            
    print(setops[-10:])
    return setops

# channel, day, seg data 생성 
def create_others():
    # 20
    channels = ['KBS', 'MBC', 'SBS', 'JTBC', 'CBS' ,  'OCN', 'TVN', 'TVCH', 'BTN', 'EBS',  'Arirang', 'JTV', 'GAME-TV', 'HBC', 'BBC',  'CNN', 'CNBC', 'CCN', 'NHK', 'ABC'] 
    days = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
    hour_bands = ['00', '01', '10', '11', '23']
    segs = ['Agriculture', 'Game']
    rows = []
    row = []
    for c in channels:
        for d in days:
            for s in segs:
                row = [c, d, s]
                rows.append(row)
                
    print(rows[:5])
    return rows

def merge_to_inventory(setops, rows):
    inven_time = 10000 # 하드코딩 시간
    invens = []
    for r in rows:
        for s in setops:
            invens.append(r + [s, inven_time])
    print(f'Inventory Length : {len(invens):,}')
    return invens

def define_schema():
    from pyspark.sql.types import StructType, StructField, StringType, LongType
    columns = [
        StructField("channel", StringType())
        , StructField("day", StringType())
        , StructField("seg", StringType())
        , StructField("setop", StringType())
        , StructField("remains", LongType())
    ]
    inven_schema = StructType(columns)
    return inven_schema

def save_inventory(invens, spark_session=spark, file_name=f'{DATA_PATH}/inven', sample_count=10000):
    inven_schema = define_schema()
    if (sample_count <= 0):
        # all data 
        rdd = spark_session.sparkContext.parallelize(invens)
    else:
        # sampling data
        rdd = spark_session.sparkContext.parallelize(invens[:sample_count])
    df = spark_session.createDataFrame(rdd, inven_schema)
    #df.write.save(path=file_name, format='csv', mode='append', sep=',')
    df.write.save(path=file_name, format=FILE_FORMAT, mode='append', sep=',')
        

In [12]:
%%time
setops = create_setops()
rows = create_others()
invens = merge_to_inventory(setops, rows)

['ST_E_1990', 'ST_E_1991', 'ST_E_1992', 'ST_E_1993', 'ST_E_1994', 'ST_E_1995', 'ST_E_1996', 'ST_E_1997', 'ST_E_1998', 'ST_E_1999']
[['KBS', 'mon', 'Agriculture'], ['KBS', 'mon', 'Game'], ['KBS', 'tue', 'Agriculture'], ['KBS', 'tue', 'Game'], ['KBS', 'wed', 'Agriculture']]
Inventory Length : 2,800,000
CPU times: user 1.71 s, sys: 184 ms, total: 1.9 s
Wall time: 1.89 s


In [15]:
%%time
# invens data 파일로 저장하기 
# gcp vm local : 12 s. csv - 90 MB, parq - 4.5 MB, 280만 rows 
SAMPLING_COUNT = int(1e7)
TABLE_NAME = 'hdfs://172.17.0.3:9000/user/root/inven'
# need file permission  
#TABLE_NAME = f'file:///home'

print(f'SAMPLING_COUNT : {SAMPLING_COUNT:,}')
print(f'TABLE_NAME : {TABLE_NAME}')
print(f'Inventory Length : {len(invens):,}')
      
save_inventory(invens, spark, TABLE_NAME, SAMPLING_COUNT)

SAMPLING_COUNT : 10,000,000
TABLE_NAME : hdfs://172.17.0.3:9000/user/root/inven
Inventory Length : 2,800,000
CPU times: user 1.04 s, sys: 94.1 ms, total: 1.14 s
Wall time: 8.3 s


In [16]:
spark.read.text(TABLE_NAME).show(5)

+--------------------+
|               value|
+--------------------+
|     PAR1�z鐨�|
|<  A   TVN...|
|0   ���� ...|
|0   ���� ...|
|0   ���� ...|
+--------------------+
only showing top 5 rows

